<a href="https://colab.research.google.com/github/mhuckvale/pals0039/blob/master/Answers_3_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<p align="center"><a href="https://www.phon.ucl.ac.uk/courses/pals0039/"><img align='center' src="https://www.phon.ucl.ac.uk/courses/pals0039/images/pals0039logo.png" title="PALS0039 Logo"></a></p>

#Exercise 3.3 Answers

(a) Load in vowels data set and prepare for learning

In [0]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
%tensorflow_version 2.x
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.utils import to_categorical

from IPython.display import display, HTML
df=pd.read_csv("https://www.phon.ucl.ac.uk/courses/pals0039/data/vowels.csv")
#df.head()
display(df.head())
display(HTML(df.head().to_html()))

(b) Convert SEX column to a number

In [0]:
df["SEX"]=df.SEX.astype('category')
df["SEX"]=df.SEX.cat.codes

(c) copy all numbers into numpy array and standardise

In [0]:
X=np.array(df.iloc[:,3:7])
print(X)
for i in range(X.shape[1]):
  X[:,i]=(X[:,i]-np.mean(X[:,i]))/np.std(X[:,i])
print(X)


(d) convert vowels to one hot coding

In [0]:
df["VOWEL"]=df.VOWEL.astype('category')
vowels=df.VOWEL.cat.codes
print(vowels[:10])
vowelcode = to_categorical(vowels)
print(vowelcode[:10])
nvowel=vowelcode.shape[1]

(e) Divide the data into train and test

In [0]:
ntrain=int(0.8*X.shape[0])
indices = np.random.permutation(X.shape[0])
Xtrain,Xtest = X[indices[:ntrain],:], X[indices[ntrain:],:]
ytrain,ytest = vowelcode[indices[:ntrain],:], vowelcode[indices[ntrain:],:]


(f) build a classifier network with configurable number of inputs and outputs

In [0]:
def build_model(numinput,numhidden,numoutput):
  model=Sequential()
  model.add(Dense(numhidden,activation='sigmoid',input_shape=(numinput,),name="hidden_layer"))
  model.add(Dense(numoutput,activation='softmax',name="output_layer"))
  sgd=SGD(learning_rate=0.1, momentum=0.9)
  model.compile(loss='categorical_crossentropy',optimizer=sgd,metrics=["accuracy"])
  return model

(g) Train the network

In [0]:
model=build_model(Xtrain.shape[1],32,nvowel);
hist=model.fit(Xtrain,ytrain,validation_data=(Xtest,ytest),epochs=100,batch_size=16,verbose=0)
#print(hist.history)
print(model.evaluate(Xtrain,ytrain,verbose=0))

(h) Plot training curves

In [0]:
plt.figure(figsize=(10,4))
plt.subplot(1,2,1)
plt.plot(hist.history['loss'])
plt.plot(hist.history['val_loss'])
plt.xlabel("Epoch")
plt.ylabel("Loss")
plt.subplot(1,2,2)
plt.plot(hist.history['accuracy'])
plt.plot(hist.history['val_accuracy'])
plt.xlabel("Epoch")
plt.ylabel("Accuracy")
plt.legend(["accuracy","val_accuracy"])
plt.show()

(i) Calculate performance on test data

In [0]:
vowelcodepred=model.predict(Xtest)
print(vowelcodepred)
voweltrue=np.argmax(ytest,axis=1)
print(voweltrue)
vowelpred=np.argmax(vowelcodepred,axis=1)
print(vowelpred)
total=ytest.shape[0]
correct=np.sum(voweltrue==vowelpred)
print("Correct %d/%d (%.1f%%)" % (correct,total,100.0*correct/total))